In [1]:
from skimage import io, color, transform
import numpy as np
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Read data set
data_dir = '/Users/kunpenghao/Desktop/Study_in_SIT/EE_628/project'
test_str = data_dir + '/data/test1/*.jpg'
train_str = data_dir + '/data/train/*.jpg'

def convert_gray(f): 
    rgb = io.imread(f)
    rgb = transform.resize(rgb, (28, 28))
    return color.rgb2gray(rgb)

test_im = io.ImageCollection(test_str, load_func = convert_gray)
train_im = io.ImageCollection(train_str, load_func = convert_gray)

train = [0 for i in range(25000)]
test = [0 for i in range(12500)]

time_start = time.time()
for i in range(len(train_im)):
    pic = train_im[i]
    train[i] = pic
for i in range(len(test_im)):
    pic = test_im[i]
    test[i] = pic
time_end = time.time()
print(time_end - time_start)

train = np.array(train)
test = np.array(test)

train = train.reshape((len(train), np.prod(train.shape[1:])))
test = test.reshape((len(test), np.prod(test.shape[1:])))

print(train.shape)
print(test.shape)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


193.05322170257568
(25000, 784)
(12500, 784)


In [3]:
# Make class for train_set
# x_train(20000), 0-9999 cat; 10000-19999 dog
# x_val(5000), 0-2499 cat; 2500-4999 dog
# y_train(20000) y_val(5000), cat = 1, dog = 0
cat_train = train[:10000]
cat_val = train[10000:12500]
dog_train = train[12500:22500]
dog_val = train[22500:]
x_train = np.zeros([20000, 784])
x_val = np.zeros([5000, 784])
x_train[:10000] = cat_train
x_train[10000:] = dog_train
x_val[:2500] = cat_val
x_val[2500:] = dog_val
y_train = np.zeros([20000, 1])
y_val = np.zeros([5000, 1])
y_train[10000:] = 1
y_val[2500:] = 1

#y_train = y_train.tolist()
#y_val = y_val.tolist()
#x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
#x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
#x_test = test.reshape(test.shape[0], 28, 28, 1)

In [4]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential

Using TensorFlow backend.


In [5]:
input_dim = x_train.shape[1]
num_class = 2

autoencoder = Sequential()
autoencoder.add(Dense(512, activation='relu'))
autoencoder.add(Dense(128, activation='relu'))
autoencoder.add(Dense(32, activation='relu'))
autoencoder.add(Dense(2, activation='sigmoid'))
autoencoder.add(Dense(32, activation='relu'))
autoencoder.add(Dense(128, activation='relu'))
autoencoder.add(Dense(512, activation='relu'))
autoencoder.add(Dense(input_dim, activation='sigmoid'))

In [6]:
autoencoder.compile(optimizer='Adadelta', loss='mse')
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_val, x_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 4s 216us/step - loss: 0.0650 - val_loss: 0.0654
Epoch 2/50
20000/20000 [==============================] - 3s 149us/step - loss: 0.0650 - val_loss: 0.0654
Epoch 3/50
20000/20000 [==============================] - 3s 156us/step - loss: 0.0649 - val_loss: 0.0653
Epoch 4/50
20000/20000 [==============================] - 3s 154us/step - loss: 0.0648 - val_loss: 0.0652
Epoch 5/50
20000/20000 [==============================] - 3s 153us/step - loss: 0.0647 - val_loss: 0.0651
Epoch 6/50
20000/20000 [==============================] - 3s 154us/step - loss: 0.0646 - val_loss: 0.0651
Epoch 7/50
20000/20000 [==============================] - 3s 160us/step - loss: 0.0646 - val_loss: 0.0650
Epoch 8/50
20000/20000 [==============================] - 3s 155us/step - loss: 0.0645 - val_loss: 0.0649
Epoch 9/50
20000/20000 [==============================] - 3s 156us/step - loss: 0.0644 - val_loss: 0.06

In [7]:
encoder = Sequential()
encoder.add(autoencoder.layers[0])
encoder.add(autoencoder.layers[1])
encoder.add(autoencoder.layers[2])
encoder.add(autoencoder.layers[3])

In [8]:
import pandas as pd
predictions = encoder.predict_classes(test)
predictions = list(predictions)
pic = []
for i in range(12500):
    pic.append(i+1)

sub = pd.DataFrame({'id': pic,'label': predictions})
sub.to_csv('./DeepAE_output.csv', index=False)